# Setup

## Mount Drive


In [167]:
from google.colab import drive
# drive.mount('/content/drive')

In [168]:
%cd drive/'My Drive'/repositories/moleculenet/notebooks

[Errno 2] No such file or directory: 'drive/My Drive/repositories/moleculenet/notebooks'
/content/drive/My Drive/repositories/moleculenet/notebooks


In [169]:
%ls

colab_grid_search_pipeline.ipynb
colab_RF_CIs_on_fingerprints_initial.ipynb
get_ecfp4_ecfp6_final.ipynb
get_original_id_smile_target_esol.ipynb
get_original_id_smile_target_freeesolv.ipynb
get_original_id_smile_target_lipophilicity.ipynb
get_protenated_from_canonical.ipynb
get_rdkit_descriptors_final.ipynb
grid_search_results_visualised.ipynb


In [170]:
%ls ../data/

esol_original_1024ecfp4_features.csv
esol_original_1024ecfp6_features.csv
esol_original_2048ecfp4_features.csv
esol_original_2048ecfp6_features.csv
esol_original.csv
esol_original_extra_features.csv
esol_original_IdSmilesLabels.csv
esol_original_rdkit_features.csv
ESOL_README
freesolv_original_1024ecfp4_features.csv
freesolv_original_1024ecfp6_features.csv
freesolv_original_2048ecfp4_features.csv
freesolv_original_2048ecfp6_features.csv
freesolv_original.csv
freesolv_original_IdSmilesLabels.csv
freesolv_original_rdkit_features.csv
FreeSolv_README
lipophilicity_original_1024ecfp4_features.csv
lipophilicity_original_1024ecfp6_features.csv
lipophilicity_original_2048ecfp4_features.csv
lipophilicity_original_2048ecfp6_features.csv
lipophilicity_original.csv
lipophilicity_original_IdSmilesLabels.csv
lipophilicity_original_rdkit_features.csv
Lipo_README


## Import modules

In [171]:
import warnings
warnings.filterwarnings('ignore')

# saving models
import json
import pickle

# standard modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# evaluating metrics
from scipy.stats import pearsonr
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import jaccard_score # Tanimoto

# model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict

# preprocessing
from sklearn.feature_selection import VarianceThreshold # to remove zero-var features
from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler

# models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.neural_network import MLPRegressor

from sklearn.kernel_ridge import KernelRidge

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern


# pipelines
# https://scikit-learn.org/stable/modules/compose.html#combining-estimators
from sklearn.pipeline import make_pipeline, Pipeline

In [172]:
import sklearn
print(sklearn.__version__)

0.22.2.post1


In [173]:
try:
    import forestci as fci
except ImportError as e:
    ! pip install forestci
    import forestci as fci

In [174]:
print('sklearn.__version__ :', sklearn.__version__)
print('fci.__version__ :', fci.__version__)

sklearn.__version__ : 0.22.2.post1
fci.__version__ : 0.4.1


## Set plotting style

In [175]:
%matplotlib inline
plt.style.use('fivethirtyeight')

plt.rcParams['axes.facecolor']='w'
#plt.rcParams['axes.linewidth']=1
plt.rcParams['axes.edgecolor']='w'
plt.rcParams['figure.facecolor']='w'
plt.rcParams['savefig.facecolor']='w'
#plt.rcParams['grid.color']='white'

# Load and Prepare Data

In [176]:
dataset = 'freesolv'
smile_type = 'original'

# READONLY
assert dataset in ['esol', 'freesolv', 'lipophilicity']
assert smile_type in ['original', 'protonated']

## Load Features

In [177]:
# original data
id_smile_target = pd.read_csv(f'../data/{dataset}_{smile_type}_IdSmilesLabels.csv', index_col=0)

# features
rdkit_features = pd.read_csv(f'../data/{dataset}_{smile_type}_rdkit_features.csv', index_col=0)

ecfp4_1024_features = pd.read_csv(f'../data/{dataset}_{smile_type}_1024ecfp4_features.csv', index_col=0)
ecfp6_1024_features = pd.read_csv(f'../data/{dataset}_{smile_type}_1024ecfp6_features.csv', index_col=0)

ecfp4_2048_features = pd.read_csv(f'../data/{dataset}_{smile_type}_2048ecfp4_features.csv', index_col=0)
ecfp6_2048_features = pd.read_csv(f'../data/{dataset}_{smile_type}_2048ecfp6_features.csv', index_col=0)

# load target
labels = id_smile_target['labels']

In [178]:
print('rdkit_features.shape:      ', rdkit_features.shape)
print('ecfp4_1024_features.shape: ', ecfp4_1024_features.shape)
print('ecfp6_1024_features.shape: ', ecfp6_1024_features.shape)
print('ecfp4_2048_features.shape: ', ecfp4_2048_features.shape)
print('ecfp6_2048_features.shape: ', ecfp6_2048_features.shape)
print('labels.shape:              ', labels.shape)

rdkit_features.shape:       (642, 200)
ecfp4_1024_features.shape:  (642, 1024)
ecfp6_1024_features.shape:  (642, 1024)
ecfp4_2048_features.shape:  (642, 2048)
ecfp6_2048_features.shape:  (642, 2048)
labels.shape:               (642,)


In [179]:
rdkit_features.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,...,fr_hdrzine,fr_hdrzone,fr_imidazole,fr_imide,fr_isocyan,fr_isothiocyan,fr_ketone,fr_ketone_Topliss,fr_lactam,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_nitro,fr_nitro_arom,fr_nitro_arom_nonortho,fr_nitroso,fr_oxazole,fr_oxime,fr_para_hydroxylation,fr_phenol,fr_phenol_noOrthoHbond,fr_phos_acid,fr_phos_ester,fr_piperdine,fr_piperzine,fr_priamide,fr_prisulfonamd,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
"4-methoxy-N,N-dimethyl-benzamide",12.428170,-3.458874,12.428170,0.519264,0.686360,179.219,166.115,179.094629,70.0,0.0,0.252836,-0.496768,0.496768,0.252836,1.384615,2.076923,2.615385,3.675580,674.590985,20.825909,19.263710,6.263710,11.392570,9.533193,3.033193,1.946749,1.946749,1.121080,1.121080,0.570798,0.570798,-1.51,110210.129799,2.108111,3.937360,2.211653,96.190689,9.636773,5.749512,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
methanesulfonyl chloride,9.855710,-4.368056,9.855710,3.074846,0.421427,114.553,111.529,113.954228,32.0,0.0,0.229212,-0.212518,0.229212,0.212518,2.000000,2.200000,2.200000,4.534785,166.212669,7.000000,5.102709,3.675135,3.250000,2.191761,3.001103,0.437848,2.532383,0.000000,0.000000,0.000000,0.000000,0.24,21.306059,1.797156,1.143107,369.351111,39.386630,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3-methylbut-1-ene,7.349537,-3.289005,7.349537,1.280324,0.412737,70.135,60.055,70.078250,30.0,0.0,0.057236,-0.102824,0.102824,0.057236,1.800000,2.600000,2.800000,5.772015,248.517785,12.654701,12.500000,2.500000,6.354059,6.000000,1.000000,0.500000,0.500000,0.125000,0.125000,0.000000,0.000000,-0.26,644.471039,0.351195,2.007380,3.740000,47.559121,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2-ethylpyrazine,7.363796,-3.006484,7.363796,0.587878,0.536795,108.144,100.080,108.068748,42.0,0.0,0.085960,-0.261224,0.261224,0.085960,1.375000,2.250000,3.125000,3.394825,411.724942,12.723615,11.894427,3.894427,7.070386,5.894427,1.894427,1.032624,1.032624,0.553812,0.553812,0.267705,0.267705,-0.92,1798.391122,1.150900,2.403302,1.025681,59.840347,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
heptan-1-ol,7.581250,-4.173333,7.581250,3.423878,0.544191,116.204,100.076,116.120115,50.0,0.0,0.210037,-0.396377,0.396377,0.210037,1.125000,1.875000,2.625000,6.416917,459.536609,20.207107,19.908248,3.908248,10.060660,9.612372,1.704124,0.727062,0.727062,0.301031,0.301031,0.119266,0.119266,-0.04,27745.345015,0.731455,6.960000,5.960000,74.289336,5.108808,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
ecfp4_1024_features.head()

,1024ecfp4-0,1024ecfp4-1,1024ecfp4-2,1024ecfp4-3,1024ecfp4-4,1024ecfp4-5,1024ecfp4-6,1024ecfp4-7,1024ecfp4-8,1024ecfp4-9,1024ecfp4-10,1024ecfp4-11,1024ecfp4-12,1024ecfp4-13,1024ecfp4-14,1024ecfp4-15,1024ecfp4-16,1024ecfp4-17,1024ecfp4-18,1024ecfp4-19,1024ecfp4-20,1024ecfp4-21,1024ecfp4-22,1024ecfp4-23,1024ecfp4-24,1024ecfp4-25,1024ecfp4-26,1024ecfp4-27,1024ecfp4-28,1024ecfp4-29,1024ecfp4-30,1024ecfp4-31,1024ecfp4-32,1024ecfp4-33,1024ecfp4-34,1024ecfp4-35,1024ecfp4-36,1024ecfp4-37,1024ecfp4-38,1024ecfp4-39,...,1024ecfp4-984,1024ecfp4-985,1024ecfp4-986,1024ecfp4-987,1024ecfp4-988,1024ecfp4-989,1024ecfp4-990,1024ecfp4-991,1024ecfp4-992,1024ecfp4-993,1024ecfp4-994,1024ecfp4-995,1024ecfp4-996,1024ecfp4-997,1024ecfp4-998,1024ecfp4-999,1024ecfp4-1000,1024ecfp4-1001,1024ecfp4-1002,1024ecfp4-1003,1024ecfp4-1004,1024ecfp4-1005,1024ecfp4-1006,1024ecfp4-1007,1024ecfp4-1008,1024ecfp4-1009,1024ecfp4-1010,1024ecfp4-1011,1024ecfp4-1012,1024ecfp4-1013,1024ecfp4-1014,1024ecfp4-1015,1024ecfp4-1016,1024ecfp4-1017,1024ecfp4-1018,1024ecfp4-1019,1024ecfp4-1020,1024ecfp4-1021,1024ecfp4-1022,1024ecfp4-1023
"4-methoxy-N,N-dimethyl-benzamide",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
methanesulfonyl chloride,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3-methylbut-1-ene,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2-ethylpyrazine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
heptan-1-ol,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [181]:
ecfp6_1024_features.head()

,1024ecfp6-0,1024ecfp6-1,1024ecfp6-2,1024ecfp6-3,1024ecfp6-4,1024ecfp6-5,1024ecfp6-6,1024ecfp6-7,1024ecfp6-8,1024ecfp6-9,1024ecfp6-10,1024ecfp6-11,1024ecfp6-12,1024ecfp6-13,1024ecfp6-14,1024ecfp6-15,1024ecfp6-16,1024ecfp6-17,1024ecfp6-18,1024ecfp6-19,1024ecfp6-20,1024ecfp6-21,1024ecfp6-22,1024ecfp6-23,1024ecfp6-24,1024ecfp6-25,1024ecfp6-26,1024ecfp6-27,1024ecfp6-28,1024ecfp6-29,1024ecfp6-30,1024ecfp6-31,1024ecfp6-32,1024ecfp6-33,1024ecfp6-34,1024ecfp6-35,1024ecfp6-36,1024ecfp6-37,1024ecfp6-38,1024ecfp6-39,...,1024ecfp6-984,1024ecfp6-985,1024ecfp6-986,1024ecfp6-987,1024ecfp6-988,1024ecfp6-989,1024ecfp6-990,1024ecfp6-991,1024ecfp6-992,1024ecfp6-993,1024ecfp6-994,1024ecfp6-995,1024ecfp6-996,1024ecfp6-997,1024ecfp6-998,1024ecfp6-999,1024ecfp6-1000,1024ecfp6-1001,1024ecfp6-1002,1024ecfp6-1003,1024ecfp6-1004,1024ecfp6-1005,1024ecfp6-1006,1024ecfp6-1007,1024ecfp6-1008,1024ecfp6-1009,1024ecfp6-1010,1024ecfp6-1011,1024ecfp6-1012,1024ecfp6-1013,1024ecfp6-1014,1024ecfp6-1015,1024ecfp6-1016,1024ecfp6-1017,1024ecfp6-1018,1024ecfp6-1019,1024ecfp6-1020,1024ecfp6-1021,1024ecfp6-1022,1024ecfp6-1023
"4-methoxy-N,N-dimethyl-benzamide",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
methanesulfonyl chloride,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3-methylbut-1-ene,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2-ethylpyrazine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
heptan-1-ol,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [182]:
ecfp4_2048_features.head()

,2048ecfp4-0,2048ecfp4-1,2048ecfp4-2,2048ecfp4-3,2048ecfp4-4,2048ecfp4-5,2048ecfp4-6,2048ecfp4-7,2048ecfp4-8,2048ecfp4-9,2048ecfp4-10,2048ecfp4-11,2048ecfp4-12,2048ecfp4-13,2048ecfp4-14,2048ecfp4-15,2048ecfp4-16,2048ecfp4-17,2048ecfp4-18,2048ecfp4-19,2048ecfp4-20,2048ecfp4-21,2048ecfp4-22,2048ecfp4-23,2048ecfp4-24,2048ecfp4-25,2048ecfp4-26,2048ecfp4-27,2048ecfp4-28,2048ecfp4-29,2048ecfp4-30,2048ecfp4-31,2048ecfp4-32,2048ecfp4-33,2048ecfp4-34,2048ecfp4-35,2048ecfp4-36,2048ecfp4-37,2048ecfp4-38,2048ecfp4-39,...,2048ecfp4-2008,2048ecfp4-2009,2048ecfp4-2010,2048ecfp4-2011,2048ecfp4-2012,2048ecfp4-2013,2048ecfp4-2014,2048ecfp4-2015,2048ecfp4-2016,2048ecfp4-2017,2048ecfp4-2018,2048ecfp4-2019,2048ecfp4-2020,2048ecfp4-2021,2048ecfp4-2022,2048ecfp4-2023,2048ecfp4-2024,2048ecfp4-2025,2048ecfp4-2026,2048ecfp4-2027,2048ecfp4-2028,2048ecfp4-2029,2048ecfp4-2030,2048ecfp4-2031,2048ecfp4-2032,2048ecfp4-2033,2048ecfp4-2034,2048ecfp4-2035,2048ecfp4-2036,2048ecfp4-2037,2048ecfp4-2038,2048ecfp4-2039,2048ecfp4-2040,2048ecfp4-2041,2048ecfp4-2042,2048ecfp4-2043,2048ecfp4-2044,2048ecfp4-2045,2048ecfp4-2046,2048ecfp4-2047
"4-methoxy-N,N-dimethyl-benzamide",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
methanesulfonyl chloride,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3-methylbut-1-ene,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2-ethylpyrazine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
heptan-1-ol,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [183]:
ecfp6_2048_features.head()

,2048ecfp6-0,2048ecfp6-1,2048ecfp6-2,2048ecfp6-3,2048ecfp6-4,2048ecfp6-5,2048ecfp6-6,2048ecfp6-7,2048ecfp6-8,2048ecfp6-9,2048ecfp6-10,2048ecfp6-11,2048ecfp6-12,2048ecfp6-13,2048ecfp6-14,2048ecfp6-15,2048ecfp6-16,2048ecfp6-17,2048ecfp6-18,2048ecfp6-19,2048ecfp6-20,2048ecfp6-21,2048ecfp6-22,2048ecfp6-23,2048ecfp6-24,2048ecfp6-25,2048ecfp6-26,2048ecfp6-27,2048ecfp6-28,2048ecfp6-29,2048ecfp6-30,2048ecfp6-31,2048ecfp6-32,2048ecfp6-33,2048ecfp6-34,2048ecfp6-35,2048ecfp6-36,2048ecfp6-37,2048ecfp6-38,2048ecfp6-39,...,2048ecfp6-2008,2048ecfp6-2009,2048ecfp6-2010,2048ecfp6-2011,2048ecfp6-2012,2048ecfp6-2013,2048ecfp6-2014,2048ecfp6-2015,2048ecfp6-2016,2048ecfp6-2017,2048ecfp6-2018,2048ecfp6-2019,2048ecfp6-2020,2048ecfp6-2021,2048ecfp6-2022,2048ecfp6-2023,2048ecfp6-2024,2048ecfp6-2025,2048ecfp6-2026,2048ecfp6-2027,2048ecfp6-2028,2048ecfp6-2029,2048ecfp6-2030,2048ecfp6-2031,2048ecfp6-2032,2048ecfp6-2033,2048ecfp6-2034,2048ecfp6-2035,2048ecfp6-2036,2048ecfp6-2037,2048ecfp6-2038,2048ecfp6-2039,2048ecfp6-2040,2048ecfp6-2041,2048ecfp6-2042,2048ecfp6-2043,2048ecfp6-2044,2048ecfp6-2045,2048ecfp6-2046,2048ecfp6-2047
"4-methoxy-N,N-dimethyl-benzamide",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
methanesulfonyl chloride,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3-methylbut-1-ene,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2-ethylpyrazine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
heptan-1-ol,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [184]:
labels.head()

id
4-methoxy-N,N-dimethyl-benzamide   -11.01
methanesulfonyl chloride            -4.87
3-methylbut-1-ene                    1.83
2-ethylpyrazine                     -5.45
heptan-1-ol                         -4.21
Name: labels, dtype: float64

In [185]:
# Establish Feature Sets

In [186]:
all_features = pd.concat([rdkit_features,
                          ecfp4_1024_features, ecfp6_1024_features,
                          ecfp4_2048_features, ecfp6_2048_features],
                         axis='columns')

In [187]:
all_features.shape

(642, 6344)

In [188]:
all_features.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,...,2048ecfp6-2008,2048ecfp6-2009,2048ecfp6-2010,2048ecfp6-2011,2048ecfp6-2012,2048ecfp6-2013,2048ecfp6-2014,2048ecfp6-2015,2048ecfp6-2016,2048ecfp6-2017,2048ecfp6-2018,2048ecfp6-2019,2048ecfp6-2020,2048ecfp6-2021,2048ecfp6-2022,2048ecfp6-2023,2048ecfp6-2024,2048ecfp6-2025,2048ecfp6-2026,2048ecfp6-2027,2048ecfp6-2028,2048ecfp6-2029,2048ecfp6-2030,2048ecfp6-2031,2048ecfp6-2032,2048ecfp6-2033,2048ecfp6-2034,2048ecfp6-2035,2048ecfp6-2036,2048ecfp6-2037,2048ecfp6-2038,2048ecfp6-2039,2048ecfp6-2040,2048ecfp6-2041,2048ecfp6-2042,2048ecfp6-2043,2048ecfp6-2044,2048ecfp6-2045,2048ecfp6-2046,2048ecfp6-2047
"4-methoxy-N,N-dimethyl-benzamide",12.428170,-3.458874,12.428170,0.519264,0.686360,179.219,166.115,179.094629,70.0,0.0,0.252836,-0.496768,0.496768,0.252836,1.384615,2.076923,2.615385,3.675580,674.590985,20.825909,19.263710,6.263710,11.392570,9.533193,3.033193,1.946749,1.946749,1.121080,1.121080,0.570798,0.570798,-1.51,110210.129799,2.108111,3.937360,2.211653,96.190689,9.636773,5.749512,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
methanesulfonyl chloride,9.855710,-4.368056,9.855710,3.074846,0.421427,114.553,111.529,113.954228,32.0,0.0,0.229212,-0.212518,0.229212,0.212518,2.000000,2.200000,2.200000,4.534785,166.212669,7.000000,5.102709,3.675135,3.250000,2.191761,3.001103,0.437848,2.532383,0.000000,0.000000,0.000000,0.000000,0.24,21.306059,1.797156,1.143107,369.351111,39.386630,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3-methylbut-1-ene,7.349537,-3.289005,7.349537,1.280324,0.412737,70.135,60.055,70.078250,30.0,0.0,0.057236,-0.102824,0.102824,0.057236,1.800000,2.600000,2.800000,5.772015,248.517785,12.654701,12.500000,2.500000,6.354059,6.000000,1.000000,0.500000,0.500000,0.125000,0.125000,0.000000,0.000000,-0.26,644.471039,0.351195,2.007380,3.740000,47.559121,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2-ethylpyrazine,7.363796,-3.006484,7.363796,0.587878,0.536795,108.144,100.080,108.068748,42.0,0.0,0.085960,-0.261224,0.261224,0.085960,1.375000,2.250000,3.125000,3.394825,411.724942,12.723615,11.894427,3.894427,7.070386,5.894427,1.894427,1.032624,1.032624,0.553812,0.553812,0.267705,0.267705,-0.92,1798.391122,1.150900,2.403302,1.025681,59.840347,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
heptan-1-ol,7.581250,-4.173333,7.581250,3.423878,0.544191,116.204,100.076,116.120115,50.0,0.0,0.210037,-0.396377,0.396377,0.210037,1.125000,1.875000,2.625000,6.416917,459.536609,20.207107,19.908248,3.908248,10.060660,9.612372,1.704124,0.727062,0.727062,0.301031,0.301031,0.119266,0.119266,-0.04,27745.345015,0.731455,6.960000,5.960000,74.289336,5.108808,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Create feature_sets dictionary with pd.Index objects (containing feature names) as its elements

### Add original features into feature_sets

In [1]:
feature_sets = {
    'RDKit': rdkit_features.columns,
    '1024ecfp-4': ecfp4_1024_features.columns,
    '1024ecfp-6': ecfp6_1024_features.columns,
    '2048ecfp-4': ecfp4_2048_features.columns,
    '2048ecfp-6': ecfp6_2048_features.columns,
}

NameError: name 'rdkit_features' is not defined

### Adding a combination of RDKit features with every feature set in the feature_sets dictionary

In [190]:
for f in ['1024ecfp-4', '1024ecfp-6', '2048ecfp-4', '2048ecfp-6']:
    feature_sets[f'{f} + RDKit'] = feature_sets[f].union(feature_sets['RDKit'])

In [191]:
list(feature_sets.keys())

['RDKit',
 '1024ecfp-4',
 '1024ecfp-6',
 '2048ecfp-4',
 '2048ecfp-6',
 '1024ecfp-4 + RDKit',
 '1024ecfp-6 + RDKit',
 '2048ecfp-4 + RDKit',
 '2048ecfp-6 + RDKit']

In [192]:
[len(feature_sets[key]) for key in feature_sets.keys()]

[200, 1024, 1024, 2048, 2048, 1224, 1224, 2248, 2248]

# Train-Test split: 90/10

**Choose to use a small subset for training**

In [193]:
# if True, use only 100 observations with 90-10 train-test-split for computational efficiency
use_small = False

**Subset the features and shuffle the data beforehand**

In [194]:
# if use_small:
#     n = 100
# else:
#     n = all_features.shape[0]

In [195]:
# np.random.seed(42)
# test = pd.Index(np.random.choice(ready_indexes, size=n//10, replace=False))
# print(test[:5], len(test))

# train = ready_indexes.difference(test)
# print(train[:5], len(train))

In [196]:
# np.random.seed(42)
# ready_indexes = pd.Index(np.random.choice(all_features.index, size=n, replace=False))
# print(ready_indexes[:5], len(ready_indexes))

In [197]:
if use_small:
    working_size = 100
else:
    working_size = all_features.shape[0]

train_test_split_gen = ShuffleSplit(n_splits=1, test_size=0.1, random_state=42).split(all_features.iloc[:working_size])
train, test = next(train_test_split_gen)

train, test = all_features.iloc[train].index, all_features.iloc[test].index

print(test[:5], len(test))
print(train[:5], len(train))

Index(['1-bromo-2-methyl-propane', '1,2,4-trichlorodibenzo-p-dioxin',
       '1-amino-9,10-anthracenedione', 'triethylphosphate',
       '2-(nitrooxy)ethan-1-ol'],
      dtype='object') 65
Index(['cyclohexanamine', 'diphenyl ether', 'ethanol', '3-methylheptane',
       '111-trifluoropropan-2-ol'],
      dtype='object') 577


In [198]:
all_features.iloc[:working_size].shape, labels.iloc[:working_size].shape

((642, 6344), (642,))

# Training

**Choose feature set to use for now, will itarate later**

In [199]:
# main feature set to use for now
f = 'RDKit'

In [200]:
features = all_features.loc[train, feature_sets[f]]
targets = labels.loc[train]

print(features.shape, targets.shape)

(577, 200) (577,)


**Choose metrics to use**

In [201]:
scoring = {'RMSE': 'neg_root_mean_squared_error',
           'MAE': 'neg_mean_absolute_error',
           'rSq': 'r2'}

## Train-Validation split: 8 to 1 (resulting in 80-10-10 in train-val-test)

**Choose the number of times for validation**

In [202]:
num_cross_val = 1

assert isinstance(num_cross_val, int)
assert 1 <= num_cross_val <= 10

**Choose split mode: random, stratified. Note:** random in this case is the K-fold split, for which the elements are shuffled in advance.

In [203]:
split_type = 'random' 

assert isinstance(split_type, str)
assert split_type in ['random', 'stratified']

In [204]:
if split_type == 'random':
    cv = ShuffleSplit(n_splits=num_cross_val, test_size=1/9, random_state=42).split(features)
elif split_type == 'stratified':
    binned = target.apply(lambda x: int(x)) # creating stratified indices
    cv = StratifiedShuffleSplit(n_splits=num_cross_val, test_size=1/9, random_state=42).split(features, binned)

# Randomized Grid Search for parameters

In [205]:
estimators = {
    'rf':  RandomForestRegressor(),
    'xgb': XGBRegressor(),
    'krr': KernelRidge()
}

params = {
    'rf': {
        'rf__n_estimators': np.arange(50, 1050, 50),
        'rf__max_features': np.arange(0.1, 1.0, 0.1)
    },
    'xgb': {
        'xgb__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5],
        'xgb__max_depth': np.arange(1, 11, 2),
        'xgb__n_estimators': np.arange(50, 550, 50),
        'xgb__subsample': [0.5, 1]
    },
    'krr': {
        'krr__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
    }
}

**Set n_iter=20 to be consistent with GP hyperparameter search**

In [206]:
all_features[feature_sets[f]].iloc[:working_size].shape

(642, 200)

In [207]:
all_features.iloc[:working_size].shape

(642, 6344)

In [208]:
%%time

best_params = {}
cv_scores = {}
test_score = {}

for f in feature_sets:
    print(f'Using {f} features...')
    features = all_features.loc[train, feature_sets[f]]

    best_params[f] = {}
    cv_scores[f] = {}
    test_score[f] = {}


    for e in estimators:
        print(f'\tRandom search optimisation for {e} estimator...')
        
        if split_type == 'random':
            cv = ShuffleSplit(n_splits=num_cross_val, test_size=1/9, random_state=42).split(features)
        elif split_type == 'stratified':
            binned = target.apply(lambda x: int(x)) # creating stratified indices
            cv = StratifiedShuffleSplit(n_splits=num_cross_val, test_size=1/9, random_state=42).split(features, binned)

        pipe = Pipeline([('zero-var-feature-remover', VarianceThreshold()), 
                        ('scaler', StandardScaler()),
                        (e, estimators[e])])

        model = RandomizedSearchCV(pipe, param_distributions=params[e], cv=cv,
                                scoring='neg_mean_squared_error',
                                refit=True, iid='False',
                                n_iter=20, n_jobs=-1,
                                verbose=0,
                                random_state=42).fit(features, targets)

        # get different metrics for the validation set
        if split_type == 'random':
            cv = ShuffleSplit(n_splits=num_cross_val, test_size=1/9, random_state=42).split(features)
        elif split_type == 'stratified':
            binned = target.apply(lambda x: int(x)) # creating stratified indices
            cv = StratifiedShuffleSplit(n_splits=num_cross_val, test_size=1/9, random_state=42).split(features, binned)

        val_results = cross_validate(estimator=model.best_estimator_,
                                     X=features, y=targets, cv=cv,
                                     scoring=scoring, n_jobs=-1)
        cv_scores[f][e] = val_results

        # record best model parameters
        best_params[f][e] = model.best_params_

        # get different metrics for the test set
        train_test_split_gen = ShuffleSplit(n_splits=1, test_size=0.1, random_state=42).split(all_features.iloc[:working_size])
        
        
        test_results = cross_validate(estimator=model.best_estimator_,
                                      X=all_features[feature_sets[f]].iloc[:working_size],
                                      y=labels.iloc[:working_size],
                                      cv=train_test_split_gen,
                                      scoring=scoring, n_jobs=-1,
                                      return_train_score=True)
        test_score[f][e] = test_results

%store best_params
%store cv_scores
%store test_score

Using RDKit features...
	Random search optimisation for rf estimator...
	Random search optimisation for xgb estimator...
[08:30:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	Random search optimisation for krr estimator...
Using 1024ecfp-4 features...
	Random search optimisation for rf estimator...
	Random search optimisation for xgb estimator...
[08:33:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	Random search optimisation for krr estimator...
Using 1024ecfp-6 features...
	Random search optimisation for rf estimator...
	Random search optimisation for xgb estimator...
[08:37:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	Random search optimisation for krr estimator...
Using 2048ecfp-4 features...
	Random search optimisation for rf estimator...
	Random search optimis

In [209]:
# retrieve 
%store -r best_params
print(best_params)

{'RDKit': {'rf': {'rf__n_estimators': 700, 'rf__max_features': 0.8}, 'xgb': {'xgb__subsample': 1, 'xgb__n_estimators': 200, 'xgb__max_depth': 3, 'xgb__learning_rate': 0.1}, 'krr': {'krr__alpha': 100}}, '1024ecfp-4': {'rf': {'rf__n_estimators': 700, 'rf__max_features': 0.6}, 'xgb': {'xgb__subsample': 0.5, 'xgb__n_estimators': 200, 'xgb__max_depth': 3, 'xgb__learning_rate': 0.3}, 'krr': {'krr__alpha': 100}}, '1024ecfp-6': {'rf': {'rf__n_estimators': 700, 'rf__max_features': 0.8}, 'xgb': {'xgb__subsample': 0.5, 'xgb__n_estimators': 200, 'xgb__max_depth': 5, 'xgb__learning_rate': 0.1}, 'krr': {'krr__alpha': 100}}, '2048ecfp-4': {'rf': {'rf__n_estimators': 700, 'rf__max_features': 0.6}, 'xgb': {'xgb__subsample': 1, 'xgb__n_estimators': 400, 'xgb__max_depth': 9, 'xgb__learning_rate': 0.1}, 'krr': {'krr__alpha': 100}}, '2048ecfp-6': {'rf': {'rf__n_estimators': 700, 'rf__max_features': 0.5}, 'xgb': {'xgb__subsample': 1, 'xgb__n_estimators': 200, 'xgb__max_depth': 3, 'xgb__learning_rate': 0.1},

In [210]:
# retrieve 
%store -r cv_scores
print(cv_scores)

{'RDKit': {'rf': {'fit_time': array([10.17717886]), 'score_time': array([0.05330682]), 'test_RMSE': array([-1.01360212]), 'test_MAE': array([-0.63979862]), 'test_rSq': array([0.92523097])}, 'xgb': {'fit_time': array([0.62504721]), 'score_time': array([0.005476]), 'test_RMSE': array([-0.81269396]), 'test_MAE': array([-0.56259009]), 'test_rSq': array([0.95193369])}, 'krr': {'fit_time': array([0.0326004]), 'score_time': array([0.0083859]), 'test_RMSE': array([-4.02414506]), 'test_MAE': array([-3.84951141]), 'test_rSq': array([-0.17851197])}}, '1024ecfp-4': {'rf': {'fit_time': array([7.72287345]), 'score_time': array([0.0600183]), 'test_RMSE': array([-2.0848009]), 'test_MAE': array([-1.33695067]), 'test_rSq': array([0.68368782])}, 'xgb': {'fit_time': array([1.8905046]), 'score_time': array([0.01131225]), 'test_RMSE': array([-1.93873746]), 'test_MAE': array([-1.34107194]), 'test_rSq': array([0.72645754])}, 'krr': {'fit_time': array([0.079247]), 'score_time': array([0.02024555]), 'test_RMSE'

In [211]:
# retrieve 
%store -r test_score
print(pd.DataFrame(pd.DataFrame(test_score).loc['xgb' ,'RDKit']))

   fit_time  score_time  test_RMSE  ...  train_MAE  test_rSq  train_rSq
0  0.699159     0.00544  -0.781063  ...  -0.204841  0.970821   0.994644

[1 rows x 8 columns]


In [212]:
type(cv_scores['RDKit']['rf']['fit_time'])

numpy.ndarray

In [213]:
float(cv_scores['RDKit']['rf']['fit_time'])

10.177178859710693

# Saving results (dictionaries) into JSON files

In [218]:
# JSON encoder for np.int64
def default_params(o):
    if isinstance(o, np.integer):
        return int(o)
    raise TypeError

In [219]:
with open(f'../results/{dataset}_{smile_type}_random_search_best_params.json', 'w') as f:
    json.dump(best_params, f, default=default_params)

In [216]:
# JSON encoder for np.float64
def default_scores(o):
    if isinstance(o, np.ndarray):
        return float(o)
    raise TypeError

In [217]:
with open(f'../results/{dataset}_{smile_type}_random_search_best_cv_scores.json', 'w') as f:
    json.dump(cv_scores, f, default=default_scores)
    
with open(f'../results/{dataset}_{smile_type}_random_search_best_test_score.json', 'w') as f:
    json.dump(test_score, f, default=default_scores)

# Saving results (dictionaries) into Pickle files

In [258]:
with open(f'../results/{dataset}_{smile_type}_random_search_best_cv_scores.pickle', 'wb') as fp:
    pickle.dump(cv_scores, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f'../results/{dataset}_{smile_type}_random_search_best_test_score.pickle', 'wb') as fp:
    pickle.dump(test_score, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'../results/{dataset}_{smile_type}_random_search_best_params.pickle', 'wb') as fp:
    pickle.dump(best_params, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'../results/{dataset}_{smile_type}_random_search_grid_params.pickle', 'wb') as f:
    pickle.dump(params, f, protocol=pickle.HIGHEST_PROTOCOL)

In [261]:
# with open(f'../results/{dataset}_{smile_type}_random_search_best_cv_scores.pickle', 'rb') as fp:
#     cv_scores = pickle.load(fp)

# with open(f'../results/{dataset}_{smile_type}_random_search_best_test_score.pickle', 'rb') as fp:
#     test_score = pickle.load(fp)

# with open(f'../results/{dataset}_{smile_type}_random_search_best_params.pickle', 'rb') as fp:
#     best_params = pickle.load(fp)

# with open(f'../results/{dataset}_{smile_type}_random_search_grid_params.pickle', 'rb') as fp:
#     params = pickle.load(fp)

In [ ]:
# cv_scores

In [ ]:
# test_score

In [250]:
# best_params

In [246]:
# params